""""Objectif"""

In [1]:
import numpy as np
import random
import pandas as pd
#data Extraction
import zipfile
#data loading
import os
#data visualization
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import load_img
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers.experimental.preprocessing import RandomZoom,RandomFlip ,RandomRotation,RandomContrast
from tensorflow.keras.models import Sequential
#Import Core Keras layers for building nueral network
from tensorflow.keras import layers,models
#import for genearting augmented data from images
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#import for adding dense layers to the neural network
from tensorflow.keras.layers import Dense,Dropout
#Import for specifing callbacks during model training (early stoping)
from tensorflow.keras.callbacks import Callback,EarlyStopping,ModelCheckpoint
#IMport for specifing the optimzer used during model training (eg .adam)
from tensorflow.keras.optimizers import Adam
#import for pre-trained neural network architechture
from tensorflow.keras.applications import MobileNetV2,vgg16
#import for creating custom data preprocessing layers 
from tensorflow.keras import models
from tensorflow.keras.layers.experimental import preprocessing


2024-10-15 14:24:44.647037: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
DATH_PATH = "/home/jihane01/Documents/Precision-Pest-Control-/agricultural-pests-image-dataset"

In [3]:
#SETTING SEEDS
#SET Seed to ensure that if we run this again we got the same result
def seed_everything(seed = 42):
    """ This function ensure that all libraires random process produce reproducible result
    """
    #seed value for Tensorflow
    tf.random.seed(seed)
    
    #seed value for numpy
    np.random.seed(seed)
    #seed value python random
    random.seed(seed)
    
    #force Tensorflow to use a single threds ,multiple  therds are potontianale for non reprudicible result
    session_comf = tf.compat.v1.ConfigProto(
        intra_op_parallelism_threads=1,
        inter_op_parallelism_threads=1,
    )
    #make sure tf use a determinstic operation whenver possible
    tf.compat.v1.set_random_seed(seed)
    
    sess = tf.compat.v1,Session(graph = tf.compat.v1.ghet_default_graph(),config = session_comf)
    tf.compat.v1.keras.backend.set_session(sess)



In [4]:
images_path = []
labels = []

for folder_names in os.listdir(DATH_PATH):
    folder_path = os.path.join(DATH_PATH ,folder_names)
    if os.path.isdir(folder_path):
        images = [img for img in os.listdir(folder_path) if img.lower().endswith((".jpg", ".png", ".jpeg"))]
        
        for images_names in images:
            image_path = os.path.join(folder_path,images_names)
            images_path.append(image_path)
            labels.append(folder_names)


data = {
    'images_path': images_path,
    'labels' : labels,
}

df =pd.DataFrame(data)

In [5]:
df

,images_path,labels
0,/home/jihane01/Documents/Precision-Pest-Contro...,bees
1,/home/jihane01/Documents/Precision-Pest-Contro...,bees
2,/home/jihane01/Documents/Precision-Pest-Contro...,bees
3,/home/jihane01/Documents/Precision-Pest-Contro...,bees
4,/home/jihane01/Documents/Precision-Pest-Contro...,bees
...,...,...
5489,/home/jihane01/Documents/Precision-Pest-Contro...,catterpillar
5490,/home/jihane01/Documents/Precision-Pest-Contro...,catterpillar
5491,/home/jihane01/Documents/Precision-Pest-Contro...,catterpillar
5492,/home/jihane01/Documents/Precision-Pest-Contro...,catterpillar


In [6]:
key = os.listdir(DATH_PATH)